In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

import env

import tda
import os
import requests
import json
import base64

Polygon.io API key loaded up.
Cboe API key loaded up.
TD Ameritrade API key loaded up.
Coinbase API key loaded up.
All credentials loaded successfully


#### Code to get new authorization key

In [2]:
# Creating the client ID variables to use in getting an access token:
client_id = env.cboe_client_id
client_secret = env.cboe_client_secret

identity_url = "https://id.livevol.com/connect/token"

encoded = base64.b64encode((client_id + ':' + client_secret).encode())
headers = {"Authorization": "Basic " + encoded.decode('ascii')}
payload = {"grant_type": "client_credentials"}

# Requesting access token
token_data = requests.post(identity_url, data=payload, headers=headers)

if token_data.status_code == 200:
    token = token_data.json()['access_token']
    if len(token) > 0:
        print("Authenticated successfully")
        # Requesting data from API
#         result = requests.get(f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}', headers={"Authorization": "Bearer " + token}, verify=False)
#         print(result.json())
else:
    print("Authentication failed")

Authenticated successfully


#### Functions to get option data

In [3]:
# Example data points used in function:

# root = 'TDOC'
# option_type = 'C'
# date = '2021-01-04'
# min_expiry = '2022-01-21'
# max_expiry = '2022-01-21'
# min_strike = '145'
# max_strike = '145'
# symbol = root


def cboe_data(root, option_type, date, min_expiry, max_expiry, min_strike, max_strike, symbol, header_code):
    
    url = f'https://api.livevol.com/v1/live/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'
    
    cboe_headers = {'Authorization': f'Bearer {header_code}'}
    
    
    # Getting the data from the API:
    stock_get = requests.get(url = url, headers = cboe_headers)
    
    stock_data = json.loads(stock_get.text)
    
    # Creating the Dataframe and cleaning up columns:
    stock_df = pd.DataFrame(stock_data['options'][0], index = [0])
    columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'option_bid', 'option_ask', 'delta', 'gamme', 'vega', 'theta', 'rho']
    
    # Using list comprehension to filter for the columns I want to keep
    stock_df = stock_df[[c for c in stock_df.columns if c in columns_to_keep]]
    
    # Adding the as of date:
    stock_df['as_of_date'] = date
    
    # Returning the df:
    
    return stock_df

In [4]:
# Function using delayed data from Cboe:

def cboe_data_delayed(root, option_type, date, min_expiry, max_expiry, min_strike, max_strike, symbol, header_code):
    
    '''
    Must use the Cboe LiveVol's API formatting for this call. This function essentially fills in the blanks for a live API call using the function arguments.
    '''
    
    url = f'https://api.livevol.com/v1/delayed/allaccess/market/option-and-underlying-quotes?root={root}&option_type={option_type}&date={date}&min_expiry={min_expiry}&max_expiry={max_expiry}&min_strike={min_strike}&max_strike={max_strike}&symbol={symbol}'
    
    cboe_headers = {'Authorization': f'Bearer {header_code}'}
    
    
    # Getting the data from the API:
    stock_get = requests.get(url = url, headers = cboe_headers)
    
    stock_data = json.loads(stock_get.text)
    
    # Creating the Dataframe and cleaning up columns:
    stock_df = pd.DataFrame(stock_data['options'][0], index = [0])
    columns_to_keep = ['root', 'expiry', 'strike', 'option_type', 'timestamp', 'option_mid', 'option_bid', 'option_ask', 'delta', 'gamme', 'vega', 'theta', 'rho']
    
    # Using list comprehension to filter for the columns I want to keep
    stock_df = stock_df[[c for c in stock_df.columns if c in columns_to_keep]]
    
    # Adding the as of date:
    stock_df['as_of_date'] = date
    
    # Returning the df:
    
    return stock_df

### Data Aquisition

- First step is to get the starting and ending prices of the option
    - The start date will be the first trading day of the year, 1/4/2021, and the last date will be the end of the previous month, 2/26/2021
- Will also retrieve the option greeks for these two dates, same option.
- Starting out looking at an ATM call.
    - Adjusted close for AAPL on 1/4/21 was 129.22, so we'll use the 130 call.

In [6]:
start_data = cboe_data('AAPL', 'C', '2021-01-04', '2021-03-19', '2021-03-19','130', '130', 'AAPL', token)

In [7]:
end_data = cboe_data('AAPL', 'C', '2021-02-26', '2021-03-19', '2021-03-19','130', '130', 'AAPL', token)

In [11]:
df = pd.concat([start_data, end_data], ignore_index = True)
df

,root,expiry,strike,option_type,timestamp,option_mid,option_bid,option_ask,delta,vega,theta,rho,as_of_date
0,AAPL,2021-03-19,130.0,C,None,8.625,8.60,8.65,0.5243,0.2320,-0.0602,0.0,2021-01-04
1,AAPL,2021-03-19,130.0,C,None,1.845,1.84,1.85,0.2616,0.0946,-0.0950,0.0,2021-02-26


In [12]:
# The calculator estimates the attribution of price changes in an option to each of the main greeks; Delta, Vega, and Theta. This is a test of that idea.